# QC Step 3 — Detection Power (Presentation Notebook)

This notebook explains **why QC sampling works**, shows the **math**, and generates a **detection power table** you can use in slides.

**What this demonstrates (v2-aligned):**
- How duplication, canaries, and spot checks combine to catch cheating.
- Why our default policy is reasonable and auditable.
- How detection changes if we tune the policy.

**Audience:** teammates, judges, and future readers. No prior statistics background required.


## How to use this in Colab

1) Open this notebook in Colab.
2) Upload the repo (zip or mount Drive).
3) Set `REPO_ROOT` below to your repo folder.

If the policy file is not found, the notebook falls back to a built‑in copy of the policy so it still runs.


In [4]:
import json
from pathlib import Path

# If running in Colab, set this to your mounted repo folder.
REPO_ROOT = Path('.').resolve()

policy_path = REPO_ROOT / "apps" / "api" / "app" / "services" / "qc" / "policy" / "qc_policy.json"

DEFAULT_POLICY = {
    "version": "1.0.0",
    "defaults": {
        "dup_rate": 0.05,
        "canary_rate": 0.01,
        "spot_rate": 0.005,
        "min_dup_packages": 1,
        "min_canaries_per_pkg": 100,
        "min_spot_items_per_pkg": 50,
        "rel_tol": 1e-4,
        "max_ulp": 2
    },
    "dispute": {
        "reexec_fraction": 0.10,
        "alpha": 0.01,
        "epsilon_threshold": 0.01
    },
    "seeding": {
        "rng": "hmac_sha256_drbg",
        "commit_phase": "gate_close",
        "job_seed_material": ["job_id", "window", "tier", "secret_epoch"],
        "provider_independence": True
    }
}

if policy_path.exists():
    policy = json.loads(policy_path.read_text())
else:
    policy = DEFAULT_POLICY

policy

{'version': '1.0.0',
 'defaults': {'dup_rate': 0.05,
  'canary_rate': 0.01,
  'spot_rate': 0.005,
  'min_dup_packages': 1,
  'min_canaries_per_pkg': 100,
  'min_spot_items_per_pkg': 50,
  'rel_tol': 0.0001,
  'max_ulp': 2},
 'dispute': {'reexec_fraction': 0.1, 'alpha': 0.01, 'epsilon_threshold': 0.01},
 'seeding': {'rng': 'hmac_sha256_drbg',
  'commit_phase': 'gate_close',
  'job_seed_material': ['job_id', 'window', 'tier', 'secret_epoch'],
  'provider_independence': True}}

## The simple detection model (plain language)

We model a provider as having an **item‑level error rate** \(\epsilon\): the fraction of items that would fail comparison against ground truth.

If we sample **n** items uniformly, the chance we miss all bad items is:

$$P(\text{miss}) = (1 - \epsilon)^n$$

So detection probability is:


$$P(\text{detect}) = 1 - (1 - \epsilon)^n$$

We combine three checks:
- **duplication** (full package rerun): catches most errors when selected
- **canaries** (known answers): per‑package sampling
- **spot re‑exec** (neutral pool): per‑package sampling

This notebook uses a **simple but clear approximation** that is easy to explain in slides.


In [ ]:
def compute_count(n_items: int, rate: float, floor: int) -> int:
    return max(floor, int(round(n_items * rate)))


def detect_prob_spot(eps: float, n_spot: int) -> float:
    return 1.0 - (1.0 - eps) ** n_spot


def detect_prob_canary(eps: float, n_can: int) -> float:
    return 1.0 - (1.0 - eps) ** n_can


def detect_prob_total(eps: float, n_spot: int, n_can: int, dup_rate: float) -> float:
    # Approx: duplication catches any mismatch, spot+canary independent
    p_spot = detect_prob_spot(eps, n_spot)
    p_can = detect_prob_canary(eps, n_can)
    p_nondup = 1.0 - (1.0 - p_spot) * (1.0 - p_can)
    return dup_rate + (1.0 - dup_rate) * p_nondup


In [1]:
# Baseline table using policy defaults
n_items = 100_000

defaults = policy["defaults"]

dup_rate = defaults["dup_rate"]
canary_rate = defaults["canary_rate"]
spot_rate = defaults["spot_rate"]

n_can = compute_count(n_items, canary_rate, defaults["min_canaries_per_pkg"])
n_spot = compute_count(n_items, spot_rate, defaults["min_spot_items_per_pkg"])

print(f"n_items={n_items}, n_can={n_can}, n_spot={n_spot}, dup_rate={dup_rate}")

for eps in [0.005, 0.01, 0.02]:
    p = detect_prob_total(eps, n_spot, n_can, dup_rate)
    print(f"eps={eps:.3%} -> P(detect) ≈ {p:.3%}")


NameError: name 'policy' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Compare a few policy variants
variants = [
    (0.02, 0.002, "dup 2% / spot 0.2%"),
    (0.05, 0.005, "dup 5% / spot 0.5% (default)"),
    (0.10, 0.01, "dup 10% / spot 1%"),
]

xs = [0.001, 0.002, 0.005, 0.01, 0.02]

plt.figure(figsize=(7, 4))
for dup_r, spot_r, label in variants:
    n_can = compute_count(n_items, canary_rate, defaults["min_canaries_per_pkg"])
    n_spot = compute_count(n_items, spot_r, defaults["min_spot_items_per_pkg"])
    ys = [detect_prob_total(eps, n_spot, n_can, dup_r) for eps in xs]
    plt.plot(xs, ys, marker="o", label=label)

plt.title("QC Detection Probability vs Error Rate")
plt.xlabel("Provider error rate (epsilon)")
plt.ylabel("P(detect)")
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


In [ ]:
# Grid table (dup_rate x spot_rate) at eps=1%

eps = 0.01

for dup_r in [0.02, 0.05, 0.10]:
    for spot_r in [0.002, 0.005, 0.01]:
        n_can = compute_count(n_items, canary_rate, defaults["min_canaries_per_pkg"])
        n_spot = compute_count(n_items, spot_r, defaults["min_spot_items_per_pkg"])
        p = detect_prob_total(eps, n_spot, n_can, dup_r)
        print(f"dup={dup_r:.3f} spot={spot_r:.3f} -> P(detect)≈{p:.3%}")
    print("-")


In [ ]:
# Small-N demo (4x3060 testbed style)
small_n_items = 2_000
n_can_small = compute_count(small_n_items, canary_rate, defaults["min_canaries_per_pkg"])
n_spot_small = compute_count(small_n_items, spot_rate, defaults["min_spot_items_per_pkg"])

print(f"small_n_items={small_n_items}, n_can={n_can_small}, n_spot={n_spot_small}")
for eps in [0.005, 0.01, 0.02]:
    p = detect_prob_total(eps, n_spot_small, n_can_small, dup_rate)
    print(f"eps={eps:.3%} -> P(detect) ≈ {p:.3%}")


## Summary

- QC uses **duplication + canaries + spot checks**.
- Even small sampling rates give strong detection probability for non‑trivial error rates.
- The policy is **deterministic**, **auditable**, and **provider‑independent**.
- This is exactly what the v2 spec requires for mechanical settlement and slashing.

**Next optional slide:** show the same chart with cost estimates (GPU minutes spent on QC) to explain the overhead trade‑off.
